# Customer Satisfaction Insights - Small LLM Exploration

This notebook covers customer sentiment insights using a cost-effective Small Language Model (SLM), focusing on deployment-friendly architectures for edge and mobile applications.

## Environment Setup

In [ ]:
!pip install torch transformers datasets flask sklearn pandas

## Load and Preprocess Data

In [ ]:

import pandas as pd
from datasets import load_dataset

# Load dataset as an example (IMDB)
dataset = load_dataset('imdb', split='train[:5000]')
df = pd.DataFrame(dataset)
df.head()


## Model Fine-tuning for Sentiment Analysis (DistilBERT)

In [ ]:

from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Data split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2)
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)

class FeedbackDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = FeedbackDataset(train_encodings, y_train.tolist())
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

from transformers import AdamW

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(2):  # fewer epochs for brevity
    epoch_loss = 0
    for batch in train_loader:
        optim.zero_grad()
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        epoch_loss += loss.item()
        loss.backward()
        optim.step()
    print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(train_loader)}")


## Evaluate Model Performance

In [ ]:

from sklearn.metrics import classification_report

test_dataset = FeedbackDataset(test_encodings, y_test.tolist())
test_loader = DataLoader(test_dataset, batch_size=16)

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
        true_labels.extend(inputs['labels'].cpu().numpy())

print(classification_report(true_labels, predictions))


## Deployment via Flask API

In [ ]:

from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    inputs = tokenizer(data['text'], return_tensors='pt', truncation=True, padding=True).to(device)
    outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=-1).item()
    sentiment = 'Positive' if pred == 1 else 'Negative'
    return jsonify({'sentiment': sentiment})

if __name__ == '__main__':
    app.run(debug=True, port=5000)


## Dockerization Instructions


Create a Dockerfile:

```Dockerfile
FROM python:3.9-slim

WORKDIR /app
COPY . .
RUN pip install torch transformers flask

EXPOSE 5000
CMD ["python", "app.py"]
```

Build and run:

```bash
docker build -t customer-satisfaction-insights .
docker run -p 5000:5000 customer-satisfaction-insights
```


## Test Flask API

In [ ]:

import requests

response = requests.post("http://localhost:5000/predict", json={"text": "This product is fantastic!"})
print(response.json())


## Conclusion

This notebook provides an end-to-end example of customer sentiment analysis using a small, cost-effective LLM, suitable for lightweight deployment scenarios like edge computing or mobile devices.